In [ ]:
from pdf2image import convert_from_path
import csv
import glob
import os
import shutil
import time

In [ ]:
# File and Directory Paths
source_dir = "source_files/"
processed_metadata_file = source_dir + "papers_processed.csv"

table_dir = "extracted_tables/"
table_metadata_file = table_dir + "tables.csv"
output_dir = table_dir + "table_images/"
code_table_dir = table_dir + "table_code/"

  
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def get_list_of_unprocessed_papers():
    csvfile_processed = open(processed_metadata_file, "r")
    spamreader = csv.reader(csvfile_processed, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
    
    list_unprocessed_papers = [row[0] for row in spamreader if row[5] == "True"]
    csvfile_processed.close()
    
    return list_unprocessed_papers

In [ ]:
"""
Converting latex code tables to png tables
unprocessed_papers - list of paper ids where table images have to be generated
amount - number of papers that should be processed
"""
def extract_data_from_papers(unprocessed_papers, amount):
    faulty_tables = []
    for i in range(amount):          
        paper_id = unprocessed_papers[i]
        paper_path = source_dir + paper_id
        print(paper_path)
        if os.path.isdir(paper_path):
            table_files = [x for x in os.listdir(paper_path) if x.startswith('FR_TAB_')]
            print(str(i) + ", " + paper_id + ": " + str(len(table_files)) + " tables")

            for table in table_files:
                try:
                    # Using pdflatex to generate pdf file
                    os.system(f"pdflatex -interaction=nonstopmode -output-directory={paper_path} {os.path.join(paper_path, table)}")

                    # Converting pdf file to an image
                    table = table.replace(".tex", ".pdf")
                    images = convert_from_path(os.path.join(paper_path, table))              
                    table = table.replace(".pdf", ".png")            
                    saved_png_path = os.path.join(paper_path, table)
                    table_id = table.replace("FR_TAB_", "")
                    images[0].save(saved_png_path, "PNG")

                    # Trimming whitespace
                    os.system(f"magick {saved_png_path} -trim {output_dir + table_id}")

                except Exception as e:
                    faulty_tables.append(table)
                    print(f"Error Type: {type(e).__name__}")
                    error_message = str(e)[:100]
                    print(f"Error Message: {error_message}")

            # Remove the paper from disk
            try:
                shutil.rmtree(source_dir + paper_id)
            except Exception as e:
                print(f"Paper {paper_id} could not be deleted.")
                print(f"Error Type: {type(e).__name__}")
                error_message = str(e)[:100]
                print(f"Error Message: {error_message}")
                      
        else:
            print("Paper could not be found on the disk")
    
    # Removing faulty tables from dataset
    remove_from_dataset(faulty_tables)
    
    # Writing changes into the csv file
    write_changes_into_csvfile(unprocessed_papers[0:amount])

# Change to_be_processed_value in the csv paper metadata file
def write_changes_into_csvfile(processed_papers):
    with open(processed_metadata_file, "r", newline='', encoding='utf-8') as input_file:
        with open(source_dir + "tmp.csv", "w", newline='', encoding='utf-8') as output_file:
            csv_reader = csv.reader(input_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            csv_writer = csv.writer(output_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
            for row in csv_reader:
                if row[0] in processed_papers:
                    row[5] = False
                csv_writer.writerow(row)
                
    # Replace old csv file with new csv file
    os.replace(source_dir + "tmp.csv", processed_metadata_file)
    
# Removes a subset of tables from the dataset (including table_image, table_code)
def remove_from_dataset(table_subset):
    table_code_files = os.listdir(code_table_dir)
    
    for table in table_subset:
        table_code = table + ".txt"
        
        if table_code in table_code_files:
            os.remove(table_code_dir + table_code)
    
    # Remove filtered tables from metadata file
    with open(table_metadata_file, "r", newline='', encoding='utf-8') as input_file:
        with open(table_dir + "tmp.csv", "w", newline='', encoding='utf-8') as output_file:
            csv_reader = csv.reader(input_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            csv_writer = csv.writer(output_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
            # Only writes rows of not deleted tables
            for row in csv_reader:
                if row[0] not in table_subset:
                    csv_writer.writerow(row)
                    
    # Replace old csv file with new csv file
    os.replace(table_dir + "tmp.csv", table_metadata_file)

In [ ]:
unprocessed_papers = get_list_of_unprocessed_papers()
print(f"Length of unprocessed papers: {len(unprocessed_papers)}")

In [ ]:
# Set number of papers that should be processed (meaning extracting table images from them)
number_of_papers_to_process = 500

# Process papers
extract_data_from_papers(unprocessed_papers, number_of_papers_to_process)

In [ ]:
import re

tables = re.findall(r"\d+, ([0-9\.]*): \d+ tables", output_message)
print(len(tables))

write_changes_into_csvfile(tables)

In [ ]:
output_message = """source_files/2306.05524
0, 2306.05524: 54 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.05524\FR_TAB_2306.05524_T
source_files/2306.09860
1, 2306.09860: 1 tables
source_files/2306.11044
2, 2306.11044: 3 tables
source_files/2307.00761
3, 2307.00761: 1 tables
source_files/2307.09498
4, 2307.09498: 2 tables
source_files/2307.13842
5, 2307.13842: 6 tables
source_files/2307.13916
6, 2307.13916: 2 tables
source_files/2308.03584
7, 2308.03584: 7 tables
source_files/2308.06668
8, 2308.06668: 1 tables
source_files/2308.07233
9, 2308.07233: 12 tables
source_files/2308.07553
10, 2308.07553: 1 tables
source_files/2308.14969
11, 2308.14969: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2308.14969\FR_TAB_2308.14969_T
source_files/2309.05472
12, 2309.05472: 16 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.05472\FR_TAB_2309.05472_T
source_files/2309.11737
13, 2309.11737: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.11737\FR_TAB_2309.11737_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.11737\FR_TAB_2309.11737_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.11737\FR_TAB_2309.11737_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.11737\FR_TAB_2309.11737_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.11737\FR_TAB_2309.11737_T
source_files/2309.13202
14, 2309.13202: 5 tables
source_files/2310.01015
15, 2310.01015: 16 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.01015\FR_TAB_2310.01015_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.01015\FR_TAB_2310.01015_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.01015\FR_TAB_2310.01015_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.01015\FR_TAB_2310.01015_T
source_files/2310.05240
16, 2310.05240: 1 tables
source_files/2310.05556
17, 2310.05556: 6 tables
source_files/2310.08754
18, 2310.08754: 1 tables
source_files/2310.10636
19, 2310.10636: 14 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.10636\FR_TAB_2310.10636_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.10636\FR_TAB_2310.10636_T
source_files/2310.16640
20, 2310.16640: 8 tables
source_files/2311.08091
21, 2311.08091: 1 tables
source_files/2311.17248
22, 2311.17248: 3 tables
source_files/2311.17950
23, 2311.17950: 12 tables
source_files/2312.00957
24, 2312.00957: 4 tables
source_files/2312.05883
25, 2312.05883: 2 tables
source_files/2312.12620
26, 2312.12620: 2 tables
source_files/2312.16704
27, 2312.16704: 3 tables
source_files/2401.08022
28, 2401.08022: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.08022\FR_TAB_2401.08022_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.08022\FR_TAB_2401.08022_T
source_files/2401.08948
29, 2401.08948: 1 tables
source_files/2401.11317
30, 2401.11317: 1 tables
source_files/2401.11455
31, 2401.11455: 10 tables
source_files/2401.13937
32, 2401.13937: 12 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.13937\FR_TAB_2401.13937_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.13937\FR_TAB_2401.13937_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.13937\FR_TAB_2401.13937_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.13937\FR_TAB_2401.13937_T
source_files/2402.05131
33, 2402.05131: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2402.05131\FR_TAB_2402.05131_T
source_files/2402.05347
34, 2402.05347: 6 tables
source_files/2402.08976
35, 2402.08976: 5 tables
source_files/2402.10359
36, 2402.10359: 2 tables
source_files/2402.18152
37, 2402.18152: 18 tables
source_files/2403.01222
38, 2403.01222: 2 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.01222\FR_TAB_2403.01222_T
source_files/2403.03193
39, 2403.03193: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.03193\FR_TAB_2403.03193_T
source_files/2403.04692
40, 2403.04692: 5 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.04692\FR_TAB_2403.04692_T
source_files/2403.05725
41, 2403.05725: 11 tables
source_files/2403.06880
42, 2403.06880: 3 tables
source_files/2403.07773
43, 2403.07773: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.07773\FR_TAB_2403.07773_T
source_files/2403.09404
44, 2403.09404: 3 tables
source_files/2403.10040
45, 2403.10040: 5 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10040\FR_TAB_2403.10040_T

Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10040\FR_TAB_2403.10040_T
source_files/2403.10534
46, 2403.10534: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10534\FR_TAB_2403.10534_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10534\FR_TAB_2403.10534_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10534\FR_TAB_2403.10534_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10534\FR_TAB_2403.10534_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10534\FR_TAB_2403.10534_T
source_files/2403.10538
47, 2403.10538: 2 tables
source_files/2403.10557
48, 2403.10557: 10 tables
source_files/2403.10562
49, 2403.10562: 4 tables
source_files/2403.10565
50, 2403.10565: 1 tables
source_files/2403.10575
51, 2403.10575: 2 tables
source_files/2403.10622
52, 2403.10622: 2 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10622\FR_TAB_2403.10622_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10622\FR_TAB_2403.10622_T
source_files/2403.10647
53, 2403.10647: 1 tables
source_files/2403.10652
54, 2403.10652: 4 tables
source_files/2403.10692
55, 2403.10692: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10692\FR_TAB_2403.10692_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10692\FR_TAB_2403.10692_T
source_files/2403.10695
56, 2403.10695: 1 tables
source_files/2403.10730
57, 2403.10730: 2 tables
source_files/2403.10735
58, 2403.10735: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10735\FR_TAB_2403.10735_T
source_files/2403.10737
59, 2403.10737: 6 tables
source_files/2403.10744
60, 2403.10744: 5 tables
source_files/2403.10753
61, 2403.10753: 6 tables
source_files/2403.10762
62, 2403.10762: 1 tables
source_files/2403.10768
63, 2403.10768: 1 tables
source_files/2403.10782
64, 2403.10782: 12 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10782\FR_TAB_2403.10782_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10782\FR_TAB_2403.10782_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10782\FR_TAB_2403.10782_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10782\FR_TAB_2403.10782_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10782\FR_TAB_2403.10782_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10782\FR_TAB_2403.10782_T
source_files/2403.10786
65, 2403.10786: 1 tables
source_files/2403.10787
66, 2403.10787: 12 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10787\FR_TAB_2403.10787_T
source_files/2403.10789
67, 2403.10789: 3 tables
source_files/2403.10790
68, 2403.10790: 7 tables
source_files/2403.10792
69, 2403.10792: 5 tables
source_files/2403.10796
70, 2403.10796: 2 tables
source_files/2403.10807
71, 2403.10807: 3 tables
source_files/2403.10819
72, 2403.10819: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10819\FR_TAB_2403.10819_T
source_files/2403.10825
73, 2403.10825: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10825\FR_TAB_2403.10825_T
source_files/2403.10826
74, 2403.10826: 5 tables
source_files/2403.10856
75, 2403.10856: 2 tables
source_files/2403.10894
76, 2403.10894: 7 tables
source_files/2403.10900
77, 2403.10900: 10 tables
source_files/2403.10917
78, 2403.10917: 1 tables
source_files/2403.10919
79, 2403.10919: 1 tables
source_files/2403.10933
80, 2403.10933: 2 tables
source_files/2403.10935
81, 2403.10935: 5 tables
source_files/2403.10958
82, 2403.10958: 1 tables
source_files/2403.10976
83, 2403.10976: 2 tables
source_files/2403.10995
84, 2403.10995: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10995\FR_TAB_2403.10995_T
source_files/2403.11000
85, 2403.11000: 2 tables
source_files/2403.11025
86, 2403.11025: 5 tables
source_files/2403.11032
87, 2403.11032: 1 tables
source_files/2403.11048
88, 2403.11048: 2 tables
source_files/2403.11051
89, 2403.11051: 3 tables
source_files/2403.11082
90, 2403.11082: 5 tables
source_files/2403.11095
91, 2403.11095: 1 tables
source_files/2403.11114
92, 2403.11114: 1 tables
source_files/2403.11136
93, 2403.11136: 3 tables
source_files/2403.11152
94, 2403.11152: 8 tables
source_files/2403.11156
95, 2403.11156: 1 tables
source_files/2403.11162
96, 2403.11162: 1 tables
source_files/2403.11164
97, 2403.11164: 1 tables
source_files/2403.11166
98, 2403.11166: 15 tables
source_files/2403.11171
99, 2403.11171: 2 tables
source_files/2403.11180
100, 2403.11180: 6 tables
source_files/2403.11184
101, 2403.11184: 6 tables
source_files/2403.11194
102, 2403.11194: 8 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11194\FR_TAB_2403.11194_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11194\FR_TAB_2403.11194_T
source_files/2403.11197
103, 2403.11197: 8 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11197\FR_TAB_2403.11197_T
source_files/2403.11203
104, 2403.11203: 6 tables
source_files/2403.11230
105, 2403.11230: 3 tables
source_files/2403.11237
106, 2403.11237: 5 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11237\FR_TAB_2403.11237_T
source_files/2403.11238
107, 2403.11238: 5 tables
source_files/2403.11254
108, 2403.11254: 5 tables
source_files/2403.11263
109, 2403.11263: 5 tables
source_files/2403.11304
110, 2403.11304: 1 tables
source_files/2403.11328
111, 2403.11328: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11328\FR_TAB_2403.11328_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11328\FR_TAB_2403.11328_T
source_files/2403.11334
112, 2403.11334: 4 tables

source_files/2403.11363
113, 2403.11363: 2 tables
source_files/2403.11373
114, 2403.11373: 7 tables
source_files/2403.11378
115, 2403.11378: 12 tables
source_files/2403.11379
116, 2403.11379: 2 tables
source_files/2403.11380
117, 2403.11380: 12 tables
source_files/2403.11402
118, 2403.11402: 4 tables
source_files/2403.11408
119, 2403.11408: 15 tables
source_files/2403.11411
120, 2403.11411: 2 tables
source_files/2403.11412
121, 2403.11412: 3 tables
source_files/2403.11413
122, 2403.11413: 3 tables
source_files/2403.11440
123, 2403.11440: 1 tables
source_files/2403.11453
124, 2403.11453: 1 tables
source_files/2403.11460
125, 2403.11460: 11 tables
source_files/2403.11480
126, 2403.11480: 4 tables
source_files/2403.11483
127, 2403.11483: 7 tables
source_files/2403.11485
128, 2403.11485: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11485\FR_TAB_2403.11485_T
source_files/2403.11496
129, 2403.11496: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11496\FR_TAB_2403.11496_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11496\FR_TAB_2403.11496_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11496\FR_TAB_2403.11496_T
source_files/2403.11509
130, 2403.11509: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11509\FR_TAB_2403.11509_T
source_files/2403.11593
131, 2403.11593: 5 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11593\FR_TAB_2403.11593_T
source_files/2403.11599
132, 2403.11599: 3 tables
source_files/2403.11611
133, 2403.11611: 5 tables
source_files/2403.11621
134, 2403.11621: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11621\FR_TAB_2403.11621_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11621\FR_TAB_2403.11621_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11621\FR_TAB_2403.11621_T
source_files/2403.11633
135, 2403.11633: 6 tables
source_files/2403.11636
136, 2403.11636: 1 tables
source_files/2403.11642
137, 2403.11642: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11642\FR_TAB_2403.11642_T
source_files/2403.11646
138, 2403.11646: 2 tables
source_files/2403.11662
139, 2403.11662: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11662\FR_TAB_2403.11662_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11662\FR_TAB_2403.11662_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11662\FR_TAB_2403.11662_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11662\FR_TAB_2403.11662_T
source_files/2403.11665
140, 2403.11665: 3 tables
source_files/2403.11671
141, 2403.11671: 2 tables
source_files/2403.11730
142, 2403.11730: 3 tables
source_files/2403.11760
143, 2403.11760: 5 tables
source_files/2403.11778
144, 2403.11778: 2 tables
source_files/2403.11803
145, 2403.11803: 6 tables
source_files/2403.11810
146, 2403.11810: 12 tables
source_files/2403.11833
147, 2403.11833: 10 tables
source_files/2403.11836
148, 2403.11836: 1 tables
source_files/2403.11870
149, 2403.11870: 6 tables
source_files/2403.11872
150, 2403.11872: 1 tables
source_files/2403.11873
151, 2403.11873: 8 tables
source_files/2403.11990
152, 2403.11990: 9 tables
source_files/2403.12004
153, 2403.12004: 7 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12004\FR_TAB_2403.12004_T
source_files/2403.12035
154, 2403.12035: 3 tables
source_files/2403.12038
155, 2403.12038: 1 tables
source_files/2304.08260
156, 2304.08260: 14 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.08260\FR_TAB_2304.08260_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.08260\FR_TAB_2304.08260_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.08260\FR_TAB_2304.08260_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.08260\FR_TAB_2304.08260_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.08260\FR_TAB_2304.08260_T
source_files/2305.17496
157, 2305.17496: 36 tables
source_files/2306.03000
158, 2306.03000: 5 tables
source_files/2307.12418
159, 2307.12418: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.12418\FR_TAB_2307.12418_T
source_files/2307.13716
160, 2307.13716: 7 tables
source_files/2308.09591
161, 2308.09591: 3 tables
source_files/2309.07439
162, 2309.07439: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.07439\FR_TAB_2309.07439_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.07439\FR_TAB_2309.07439_T
source_files/2309.13173
163, 2309.13173: 2 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2309.13173\FR_TAB_2309.13173_T
source_files/2310.14855
164, 2310.14855: 9 tables
source_files/2311.02778
165, 2311.02778: 3 tables
source_files/2311.06455
166, 2311.06455: 3 tables
source_files/2311.10081
167, 2311.10081: 7 tables
source_files/2311.15056
168, 2311.15056: 2 tables
source_files/2311.16191
169, 2311.16191: 10 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2311.16191\FR_TAB_2311.16191_T
source_files/2311.16974
170, 2311.16974: 2 tables
source_files/2312.10467
171, 2312.10467: 8 tables
source_files/2312.13316
172, 2312.13316: 7 tables
source_files/2401.00865
173, 2401.00865: 5 tables
source_files/2401.07603
174, 2401.07603: 5 tables
source_files/2401.08846
175, 2401.08846: 4 tables
source_files/2401.16205
176, 2401.16205: 1 tables
source_files/2401.17319
177, 2401.17319: 5 tables
source_files/2402.10109
178, 2402.10109: 8 tables
source_files/2402.11453
179, 2402.11453: 2 tables
source_files/2403.01811
180, 2403.01811: 4 tables
source_files/2403.06097
181, 2403.06097: 4 tables
source_files/2403.08807
182, 2403.08807: 3 tables
source_files/2403.09254
183, 2403.09254: 11 tables
source_files/2403.09362
184, 2403.09362: 3 tables
source_files/2403.10554
185, 2403.10554: 2 tables
source_files/2403.10904
186, 2403.10904: 3 tables
source_files/2403.11206
187, 2403.11206: 10 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11206\FR_TAB_2403.11206_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.11206\FR_TAB_2403.11206_T
source_files/2403.11366
188, 2403.11366: 2 tables
source_files/2403.11461
189, 2403.11461: 3 tables
source_files/2403.11492
190, 2403.11492: 2 tables
source_files/2403.11762
191, 2403.11762: 1 tables
source_files/2403.12045
192, 2403.12045: 4 tables
source_files/2403.12077
193, 2403.12077: 2 tables
source_files/2403.12083
194, 2403.12083: 5 tables
source_files/2403.12094
195, 2403.12094: 12 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12094\FR_TAB_2403.12094_T

Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12094\FR_TAB_2403.12094_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12094\FR_TAB_2403.12094_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12094\FR_TAB_2403.12094_T
source_files/2403.12106
196, 2403.12106: 1 tables
source_files/2403.12117
197, 2403.12117: 3 tables
source_files/2403.12202
198, 2403.12202: 11 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12202\FR_TAB_2403.12202_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12202\FR_TAB_2403.12202_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12202\FR_TAB_2403.12202_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12202\FR_TAB_2403.12202_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12202\FR_TAB_2403.12202_T
source_files/2403.12231
199, 2403.12231: 18 tables
source_files/2403.12252
200, 2403.12252: 4 tables
source_files/2403.12274
201, 2403.12274: 1 tables
source_files/2403.12280
202, 2403.12280: 8 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12280\FR_TAB_2403.12280_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12280\FR_TAB_2403.12280_T
source_files/2403.12307
203, 2403.12307: 9 tables
source_files/2403.12323
204, 2403.12323: 10 tables
source_files/2403.12364
205, 2403.12364: 1 tables
source_files/2403.12368
206, 2403.12368: 3 tables
source_files/2403.12378
207, 2403.12378: 1 tables
source_files/2403.12382
208, 2403.12382: 3 tables
source_files/2403.12389
209, 2403.12389: 9 tables
source_files/2403.12391
210, 2403.12391: 6 tables
source_files/2403.12392
211, 2403.12392: 15 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12392\FR_TAB_2403.12392_T
source_files/2403.12399
212, 2403.12399: 3 tables
source_files/2403.12413
213, 2403.12413: 9 tables
source_files/2403.12436
214, 2403.12436: 2 tables
source_files/2403.12446
215, 2403.12446: 2 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12446\FR_TAB_2403.12446_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12446\FR_TAB_2403.12446_T
source_files/2403.12460
216, 2403.12460: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12460\FR_TAB_2403.12460_T
source_files/2403.12462
217, 2403.12462: 2 tables
source_files/2403.12471
218, 2403.12471: 1 tables
source_files/2403.12493
219, 2403.12493: 2 tables
source_files/2403.12499
220, 2403.12499: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12499\FR_TAB_2403.12499_T
source_files/2403.12501
221, 2403.12501: 1 tables
source_files/2403.12523
222, 2403.12523: 2 tables
source_files/2403.12537
223, 2403.12537: 2 tables
source_files/2403.12556
224, 2403.12556: 10 tables
source_files/2403.12562
225, 2403.12562: 1 tables
source_files/2403.12568
226, 2403.12568: 3 tables
source_files/2403.12577
227, 2403.12577: 4 tables
source_files/2403.12581
228, 2403.12581: 5 tables
source_files/2403.12585
229, 2403.12585: 1 tables
source_files/2403.12606
230, 2403.12606: 1 tables
source_files/2403.12609
231, 2403.12609: 6 tables
source_files/2403.12618
232, 2403.12618: 5 tables
source_files/2403.12666
233, 2403.12666: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12666\FR_TAB_2403.12666_T
source_files/2403.12693
234, 2403.12693: 9 tables
source_files/2403.12739
235, 2403.12739: 2 tables
source_files/2403.12744
236, 2403.12744: 11 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12744\FR_TAB_2403.12744_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12744\FR_TAB_2403.12744_T
source_files/2403.12748
237, 2403.12748: 2 tables
source_files/2403.12749
238, 2403.12749: 10 tables
source_files/2403.12776
239, 2403.12776: 4 tables
source_files/2403.12809
240, 2403.12809: 17 tables
source_files/2403.12834
241, 2403.12834: 2 tables
source_files/2403.12933
242, 2403.12933: 1 tables
source_files/2403.12953
243, 2403.12953: 12 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12953\FR_TAB_2403.12953_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12953\FR_TAB_2403.12953_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12953\FR_TAB_2403.12953_T
source_files/2403.12961
244, 2403.12961: 4 tables
source_files/2301.08832
245, 2301.08832: 9 tables
source_files/2303.02237
246, 2303.02237: 7 tables
source_files/2303.05100
247, 2303.05100: 2 tables
source_files/2304.09067
248, 2304.09067: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.09067\FR_TAB_2304.09067_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.09067\FR_TAB_2304.09067_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.09067\FR_TAB_2304.09067_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2304.09067\FR_TAB_2304.09067_T
source_files/2305.09098
249, 2305.09098: 1 tables
source_files/2305.11514
250, 2305.11514: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2305.11514\FR_TAB_2305.11514_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2305.11514\FR_TAB_2305.11514_T
source_files/2305.14456
251, 2305.14456: 18 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2305.14456\FR_TAB_2305.14456_T
source_files/2306.02371
252, 2306.02371: 7 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.02371\FR_TAB_2306.02371_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2306.02371\FR_TAB_2306.02371_T
source_files/2307.04536
253, 2307.04536: 2 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.04536\FR_TAB_2307.04536_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.04536\FR_TAB_2307.04536_T
source_files/2307.05113
254, 2307.05113: 21 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.05113\FR_TAB_2307.05113_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.05113\FR_TAB_2307.05113_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.05113\FR_TAB_2307.05113_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.05113\FR_TAB_2307.05113_T

Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.05113\FR_TAB_2307.05113_T
source_files/2308.00221
255, 2308.00221: 16 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2308.00221\FR_TAB_2308.00221_T
source_files/2308.03001
256, 2308.03001: 2 tables
source_files/2308.08739
257, 2308.08739: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2308.08739\FR_TAB_2308.08739_T
source_files/2309.02706
258, 2309.02706: 14 tables
source_files/2309.03320
259, 2309.03320: 7 tables
source_files/2309.04461
260, 2309.04461: 2 tables
source_files/2309.09783
261, 2309.09783: 7 tables
source_files/2309.17223
262, 2309.17223: 8 tables
source_files/2310.04086
263, 2310.04086: 3 tables
source_files/2310.04268
264, 2310.04268: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.04268\FR_TAB_2310.04268_T
source_files/2312.04539
265, 2312.04539: 5 tables
source_files/2312.09008
266, 2312.09008: 5 tables
source_files/2312.13016
267, 2312.13016: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2312.13016\FR_TAB_2312.13016_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2312.13016\FR_TAB_2312.13016_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2312.13016\FR_TAB_2312.13016_T
source_files/2312.16244
268, 2312.16244: 8 tables
source_files/2312.17649
269, 2312.17649: 3 tables
source_files/2401.10298
270, 2401.10298: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2401.10298\FR_TAB_2401.10298_T
source_files/2401.11467
271, 2401.11467: 2 tables
source_files/2403.01233
272, 2403.01233: 4 tables
source_files/2403.06225
273, 2403.06225: 4 tables
source_files/2403.06866
274, 2403.06866: 1 tables
source_files/2403.07858
275, 2403.07858: 7 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.07858\FR_TAB_2403.07858_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.07858\FR_TAB_2403.07858_T
source_files/2403.08381
276, 2403.08381: 4 tables
source_files/2403.08505
277, 2403.08505: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.08505\FR_TAB_2403.08505_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.08505\FR_TAB_2403.08505_T
source_files/2403.09380
278, 2403.09380: 2 tables
source_files/2403.10099
279, 2403.10099: 3 tables
source_files/2403.10297
280, 2403.10297: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10297\FR_TAB_2403.10297_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10297\FR_TAB_2403.10297_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.10297\FR_TAB_2403.10297_T
source_files/2403.11310
281, 2403.11310: 16 tables
source_files/2403.12245
282, 2403.12245: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12245\FR_TAB_2403.12245_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12245\FR_TAB_2403.12245_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12245\FR_TAB_2403.12245_T
source_files/2403.12267
283, 2403.12267: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12267\FR_TAB_2403.12267_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12267\FR_TAB_2403.12267_T
source_files/2403.12894
284, 2403.12894: 5 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.12894\FR_TAB_2403.12894_T
source_files/2403.12969
285, 2403.12969: 9 tables
source_files/2403.12980
286, 2403.12980: 15 tables
source_files/2403.13010
287, 2403.13010: 15 tables
source_files/2403.13013
288, 2403.13013: 12 tables
source_files/2403.13064
289, 2403.13064: 9 tables
source_files/2403.13125
290, 2403.13125: 2 tables
source_files/2403.13135
291, 2403.13135: 5 tables
source_files/2403.13190
292, 2403.13190: 4 tables
source_files/2403.13193
293, 2403.13193: 4 tables
source_files/2403.13195
294, 2403.13195: 7 tables
source_files/2403.13208
295, 2403.13208: 3 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13208\FR_TAB_2403.13208_T
source_files/2403.13236
296, 2403.13236: 1 tables
source_files/2403.13240
297, 2403.13240: 12 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13240\FR_TAB_2403.13240_T
source_files/2403.13242
298, 2403.13242: 6 tables
source_files/2403.13253
299, 2403.13253: 11 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13253\FR_TAB_2403.13253_T
source_files/2403.13258
300, 2403.13258: 9 tables
source_files/2403.13272
301, 2403.13272: 3 tables

source_files/2403.13286
302, 2403.13286: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13286\FR_TAB_2403.13286_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13286\FR_TAB_2403.13286_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13286\FR_TAB_2403.13286_T
source_files/2403.13291
303, 2403.13291: 6 tables
source_files/2403.13294
304, 2403.13294: 5 tables
source_files/2403.13296
305, 2403.13296: 2 tables
source_files/2403.13312
306, 2403.13312: 2 tables
source_files/2403.13319
307, 2403.13319: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13319\FR_TAB_2403.13319_T
source_files/2403.13343
308, 2403.13343: 5 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13343\FR_TAB_2403.13343_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13343\FR_TAB_2403.13343_T
source_files/2403.13350
309, 2403.13350: 2 tables
source_files/2403.13375
310, 2403.13375: 8 tables
source_files/2403.13380
311, 2403.13380: 1 tables
source_files/2403.13384
312, 2403.13384: 1 tables
source_files/2403.13405
313, 2403.13405: 9 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13405\FR_TAB_2403.13405_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13405\FR_TAB_2403.13405_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13405\FR_TAB_2403.13405_T
source_files/2403.13417
314, 2403.13417: 5 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13417\FR_TAB_2403.13417_T
source_files/2403.13429
315, 2403.13429: 2 tables
source_files/2403.13440
316, 2403.13440: 2 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13440\FR_TAB_2403.13440_T
source_files/2403.13455
317, 2403.13455: 2 tables
source_files/2403.13468
318, 2403.13468: 5 tables
source_files/2403.13523
319, 2403.13523: 3 tables
source_files/2403.13528
320, 2403.13528: 12 tables
source_files/2403.13548
321, 2403.13548: 6 tables
source_files/2403.13551
322, 2403.13551: 6 tables
source_files/2403.13590
323, 2403.13590: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13590\FR_TAB_2403.13590_T
source_files/2403.13601
324, 2403.13601: 4 tables
source_files/2403.13647
325, 2403.13647: 3 tables
source_files/2403.13654
326, 2403.13654: 8 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13654\FR_TAB_2403.13654_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13654\FR_TAB_2403.13654_T
source_files/2403.13666
327, 2403.13666: 3 tables
source_files/2403.13672
328, 2403.13672: 5 tables
source_files/2403.13683
329, 2403.13683: 6 tables
source_files/2403.13707
330, 2403.13707: 1 tables
source_files/2403.13729
331, 2403.13729: 8 tables
source_files/2403.13736
332, 2403.13736: 2 tables
source_files/2403.13743
333, 2403.13743: 5 tables
source_files/2403.13761
334, 2403.13761: 8 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2403.13761\FR_TAB_2403.13761_T
source_files/2403.13785
335, 2403.13785: 1 tables
source_files/2403.13804
336, 2403.13804: 2 tables
source_files/2303.03757
337, 2303.03757: 4 tables
source_files/2303.12157
338, 2303.12157: 3 tables
source_files/2303.15662
339, 2303.15662: 2 tables
source_files/2305.09592
340, 2305.09592: 6 tables
source_files/2306.00010
341, 2306.00010: 1 tables
source_files/2306.04998
342, 2306.04998: 1 tables
source_files/2306.05042
343, 2306.05042: 1 tables
source_files/2307.06029
344, 2307.06029: 6 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2307.06029\FR_TAB_2307.06029_T
source_files/2307.13122
345, 2307.13122: 4 tables
source_files/2308.05342
346, 2308.05342: 4 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2308.05342\FR_TAB_2308.05342_T
source_files/2309.03378
347, 2309.03378: 4 tables
source_files/2309.06030
348, 2309.06030: 2 tables
source_files/2310.06155
349, 2310.06155: 15 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.06155\FR_TAB_2310.06155_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.06155\FR_TAB_2310.06155_T
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2310.06155\FR_TAB_2310.06155_T
source_files/2310.06778
350, 2310.06778: 6 tables
"""